Extracting User's Google Calendar

In [1]:
import methods 
import formats
import json

google_calendar = methods.get_google_calendar_service()
calendar = methods.extractCalendar(google_calendar)
client = methods.set_up_ChatGPT(calendar)

Here are the events for today:
1. EECS 545: Machine Learning from 9:00 AM to 10:30 AM
2. Run from 10:00 AM to 11:00 AM

Pass calendar and prompt to GPT 

Query

In [2]:
query = input()
print(query)

Move my run to 1pm tomorrow


In [3]:
prompt2 = "Extract intent from the following query meant for a virtual calendar assitant - " + query + ". Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2}],
    stream=True,
)
intent = ''
for chunk in stream:
    intent += chunk.choices[0].delta.content or ""
print(intent)   

moveEvent


In [4]:
if intent == 'moveEvent':
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': query}],
        functions = formats.moveEvent_format,
    )


    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)
    entities = dict(json_response)
    methods.moveEvent(entities)


{'name': 'run', 'time': '1:00 PM', 'new_date': '2024-03-19'}
Please specify the time to move the event to.


In [5]:
if intent == 'createEvent':
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': query}],
        functions = formats.createEvent_format,
    )

    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)
    entities = dict(json_response)
    methods.createEvent(entities)

In [6]:
#TODO: Add deleteEvent, listEvents, updateEvent, RecurringEvents intents 

In [7]:
if intent == "checkSchedule":
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions= formats.checkSchedule_format,
    )
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    entities = dict(json_response)
    st.write("Entities: ", entities)
    methods.checkSchedule(entities)
